In [5]:
import pandas as pd

In [7]:
df = pd.read_csv('C:/Users/ASUS/Downloads/ecommerce_returns_synthetic_data.csv')
df

,Order_ID,Product_ID,User_ID,Order_Date,Return_Date,Product_Category,Product_Price,Order_Quantity,Return_Reason,Return_Status,Days_to_Return,User_Age,User_Gender,User_Location,Payment_Method,Shipping_Method,Discount_Applied
0,ORD00000000,PROD00000000,USER00000000,05-08-2023,26-08-2024,Clothing,411.59,3,Changed mind,Returned,387,58,Male,City54,Debit Card,Next-Day,45.27
1,ORD00000001,PROD00000001,USER00000001,09-10-2023,09-11-2023,Books,288.88,3,Wrong item,Returned,31,68,Female,City85,Credit Card,Express,47.79
2,ORD00000002,PROD00000002,USER00000002,06-05-2023,NaN,Toys,390.03,5,NaN,Not Returned,0,22,Female,City30,Debit Card,Next-Day,26.64
3,ORD00000003,PROD00000003,USER00000003,29-08-2024,NaN,Toys,401.09,3,NaN,Not Returned,0,40,Male,City95,PayPal,Next-Day,15.37
4,ORD00000004,PROD00000004,USER00000004,16-01-2023,NaN,Books,110.09,4,NaN,Not Returned,0,34,Female,City80,Gift Card,Standard,16.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,ORD00009995,PROD00009995,USER00009995,20-10-2023,NaN,Home,142.50,4,NaN,Not Returned,0,37,Male,City40,PayPal,Standard,34.27
9996,ORD00009996,PROD00009996,USER00009996,25-02-2023,NaN,Electronics,484.63,3,NaN,Not Returned,0,69,Male,City62,Debit Card,Express,25.44
9997,ORD00009997,PROD00009997,USER00009997,10-05-2024,NaN,Toys,386.57,5,NaN,Not Returned,0,46,Male,City74,Credit Card,Next-Day,12.67
9998,ORD00009998,PROD00009998,USER00009998,13-02-2024,NaN,Toys,129.22,1,NaN,Not Returned,0,34,Female,City34,Gift Card,Express,49.97


In [9]:
df['Order_Date'] = pd.to_datetime(df['Order_Date'], format='%d-%m-%Y')
df['Return_Date'] = pd.to_datetime(df['Return_Date'], format='%d-%m-%Y', errors='coerce')

In [11]:
df['Is_Returned'] = df['Return_Status'].apply(lambda x: 1 if x == 'Returned' else 0)

In [13]:
bins = [0, 25, 35, 50, 100]
labels = ['<25', '25-35', '35-50', '50+']
df['Age_Group'] = pd.cut(df['User_Age'], bins=bins, labels=labels)

In [53]:
df_encoded = pd.get_dummies(df, columns=[
    'Product_Category', 'Payment_Method', 'Shipping_Method',
    'User_Gender', 'User_Location', 'Age_Group'  
], drop_first=True)

In [27]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score

In [55]:
X = df_encoded.drop(columns=[
    'Is_Returned', 'Return_Status', 'Return_Date', 'Return_Reason',
    'Order_ID', 'User_ID', 'Product_ID', 'Order_Date'
])
y = df_encoded['Is_Returned']

In [57]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [59]:
df_encoded['Order_Year'] = df['Order_Date'].dt.year
df_encoded['Order_Month'] = df['Order_Date'].dt.month
df_encoded['Order_Weekday'] = df['Order_Date'].dt.weekday

In [61]:
df_encoded['Return_Year'] = df['Return_Date'].dt.year
df_encoded['Return_Month'] = df['Return_Date'].dt.month
df_encoded['Return_Weekday'] = df['Return_Date'].dt.weekday

In [65]:
print(X.dtypes[X.dtypes == 'object'])

Series([], dtype: object)


In [63]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [67]:
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, model.predict_proba(X_test)[:, 1]))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1009
           1       1.00      1.00      1.00       991

    accuracy                           1.00      2000
   macro avg       1.00      1.00      1.00      2000
weighted avg       1.00      1.00      1.00      2000

ROC AUC: 1.0


In [69]:
df['Return_Probability'] = model.predict_proba(X)[:, 1]

In [71]:
high_risk = df[df['Return_Probability'] > 0.6]
high_risk.to_csv('high_risk_products.csv', index=False)